In [1]:
import pandas as pd
import numpy as np
import dateutil
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import altair as alt
from vega_datasets import data
from plotly.subplots import make_subplots
import plotly.express as px
import json

# only in jupyter notebook need plotly offline mode
# I still got some error when using Jupyterlab
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [2]:
# loading data
total_confirmed = pd.read_csv(
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv', encoding='utf-8', na_values=None)
# #Replace US with United States
total_confirmed.replace(
    to_replace='US', value='United States', regex=True, inplace=True)

total_death = pd.read_csv(
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv', encoding='utf-8', na_values=None)
total_death.replace(to_replace='US', value='United States',
                    regex=True, inplace=True)

total_recovered = pd.read_csv(
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv', encoding='utf-8', na_values=None)
total_recovered.replace(
    to_replace='US', value='United States', regex=True, inplace=True)

In [3]:
total_recovered[total_recovered["Country/Region"] == "United States"]

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
245,NaN,United States,40.0,-100.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 1. Get country-grouped and global time series data frame


#### grouped total confiremed data


In [4]:
grouped_total_confirmed = total_confirmed[["Country/Region", total_confirmed.columns[-1]]].groupby(
    "Country/Region").sum().sort_values(by=total_confirmed.columns[-1], ascending=True)

grouped_total_confirmed.reset_index(inplace=True)

grouped_total_confirmed.columns = ["Country/Region", "confirmed"]

grouped_total_confirmed.replace(
    to_replace='US', value='United States', regex=True, inplace=True)

In [5]:
grouped_total_confirmed

,Country/Region,confirmed
0,"Korea, North",1
1,MS Zaandam,9
2,Antarctica,11
3,Holy See,29
4,Winter Olympics 2022,535
...,...,...
196,Brazil,37076053
197,Germany,38249060
198,France,39866718
199,India,44690738


In [6]:
# Chart.js can't plot dataframe object, so we need to change some to list
barplot_confirmed_values = grouped_total_confirmed["confirmed"].values.tolist()
country_names = grouped_total_confirmed["Country/Region"].values.tolist()
country_names

['Korea, North',
 'MS Zaandam',
 'Antarctica',
 'Holy See',
 'Winter Olympics 2022',
 'Diamond Princess',
 'Summer Olympics 2020',
 'Tuvalu',
 'Kiribati',
 'Nauru',
 'Palau',
 'Sao Tome and Principe',
 'Saint Kitts and Nevis',
 'Chad',
 'Sierra Leone',
 'Liberia',
 'Guinea-Bissau',
 'Comoros',
 'Antigua and Barbuda',
 'Niger',
 'Saint Vincent and the Grenadines',
 'Eritrea',
 'Yemen',
 'Vanuatu',
 'Gambia',
 'Central African Republic',
 'Marshall Islands',
 'Nicaragua',
 'Djibouti',
 'Dominica',
 'Monaco',
 'Samoa',
 'Tonga',
 'Equatorial Guinea',
 'Tajikistan',
 'South Sudan',
 'Grenada',
 'Liechtenstein',
 'Burkina Faso',
 'Timor-Leste',
 'San Marino',
 'Micronesia',
 'Solomon Islands',
 'Congo (Brazzaville)',
 'Somalia',
 'Benin',
 'Saint Lucia',
 'Mali',
 'Haiti',
 'Lesotho',
 'Bahamas',
 'Guinea',
 'Togo',
 'Tanzania',
 'Papua New Guinea',
 'Andorra',
 'Gabon',
 'Seychelles',
 'Burundi',
 'Syria',
 'Bhutan',
 'Cabo Verde',
 'Mauritania',
 'Sudan',
 'Madagascar',
 'Fiji',
 'Belize'

In [7]:
# global time series confiremed data frame
global_confirmed_timeseries = pd.DataFrame(
    total_confirmed[total_confirmed.columns[4:]].sum())
global_confirmed_timeseries.reset_index(inplace=True)
global_confirmed_timeseries.columns = ['date', 'total confirmed']

# global daily new cases = global daily confirmed at date (t) -  global daily confirmed at date (t-1)
global_confirmed_timeseries["daily new cases"] = global_confirmed_timeseries["total confirmed"] - \
    global_confirmed_timeseries['total confirmed'].shift()
global_confirmed_timeseries = global_confirmed_timeseries.fillna(0)
global_confirmed_timeseries

,date,total confirmed,daily new cases
0,1/22/20,557,0.0
1,1/23/20,657,100.0
2,1/24/20,944,287.0
3,1/25/20,1437,493.0
4,1/26/20,2120,683.0
...,...,...,...
1138,3/5/23,676024901,56126.0
1139,3/6/23,676082941,58040.0
1140,3/7/23,676213378,130437.0
1141,3/8/23,676392824,179446.0


#### grouped total recovered data


In [8]:
grouped_total_recovered = total_recovered[["Country/Region", total_recovered.columns[-1]]].groupby(
    "Country/Region").sum().sort_values(by=total_recovered.columns[-1], ascending=False)

grouped_total_recovered.reset_index(inplace=True)

grouped_total_recovered.columns = ["Country/Region", "recovered"]

grouped_total_recovered.replace(
    to_replace="US", value="United States", regex=True, inplace=True)

In [9]:
# chart.js can't plot dataframe object, so we need to change some to list
barplot_recovered_values = grouped_total_recovered["recovered"].values.tolist()
country_names = grouped_total_recovered["Country/Region"].values.tolist()

In [10]:
# global time series recovered data frame
global_recovered_timeseries = pd.DataFrame(
    total_recovered[total_recovered.columns[4:]].sum())
global_recovered_timeseries.reset_index(inplace=True)
global_recovered_timeseries.columns = ["date", "total recovered"]

global_recovered_timeseries

,date,total recovered
0,1/22/20,30
1,1/23/20,32
2,1/24/20,39
3,1/25/20,42
4,1/26/20,56
...,...,...
1138,3/5/23,0
1139,3/6/23,0
1140,3/7/23,0
1141,3/8/23,0


In [11]:
global_recovered_timeseries["daily new recovered"] = global_recovered_timeseries['total recovered'] - \
    global_recovered_timeseries['total recovered'].shift()

global_recovered_timeseries = global_recovered_timeseries.fillna(0)

global_recovered_timeseries

,date,total recovered,daily new recovered
0,1/22/20,30,0.0
1,1/23/20,32,2.0
2,1/24/20,39,7.0
3,1/25/20,42,3.0
4,1/26/20,56,14.0
...,...,...,...
1138,3/5/23,0,0.0
1139,3/6/23,0,0.0
1140,3/7/23,0,0.0
1141,3/8/23,0,0.0


#### grouping the data by each country to get total confirmed cases


In [12]:
grouped_total_death = pd.DataFrame(total_death[total_death.columns[4:]].sum())
grouped_total_death.reset_index(inplace=True)
grouped_total_death.columns = ["Country/Region", "deaths"]
grouped_total_death.replace(
    to_replace="US", value="United States", regex=True, inplace=True)

grouped_total_death

,Country/Region,deaths
0,1/22/20,17
1,1/23/20,18
2,1/24/20,26
3,1/25/20,42
4,1/26/20,56
...,...,...
1138,3/5/23,6877749
1139,3/6/23,6878115
1140,3/7/23,6879038
1141,3/8/23,6880483


In [13]:
# Chart.js can't plot dataframe object, so we need to change some to list
barplot_death_values = grouped_total_death["deaths"].values.tolist()
global_death_timeseries = total_death[total_death.columns[4:]].sum()

global_death_timeseries

1/22/20         17
1/23/20         18
1/24/20         26
1/25/20         42
1/26/20         56
            ...   
3/5/23     6877749
3/6/23     6878115
3/7/23     6879038
3/8/23     6880483
3/9/23     6881802
Length: 1143, dtype: int64

In [14]:
# global time series death data frame
global_death_timeseries = pd.DataFrame(
    total_death[total_death.columns[4:]].sum())
global_death_timeseries.reset_index(inplace=True)
global_death_timeseries.columns = ["date", "total deaths"]

global_death_timeseries

,date,total deaths
0,1/22/20,17
1,1/23/20,18
2,1/24/20,26
3,1/25/20,42
4,1/26/20,56
...,...,...
1138,3/5/23,6877749
1139,3/6/23,6878115
1140,3/7/23,6879038
1141,3/8/23,6880483


In [15]:
# global daily deaths = global daily deaths at date(t) - global daily deaths at date(t-1)
global_death_timeseries["daily new deaths"] = global_death_timeseries["total deaths"] - \
    global_death_timeseries["total deaths"].shift()
global_death_timeseries = global_death_timeseries.fillna(0)
global_death_timeseries

,date,total deaths,daily new deaths
0,1/22/20,17,0.0
1,1/23/20,18,1.0
2,1/24/20,26,8.0
3,1/25/20,42,16.0
4,1/26/20,56,14.0
...,...,...,...
1138,3/5/23,6877749,148.0
1139,3/6/23,6878115,366.0
1140,3/7/23,6879038,923.0
1141,3/8/23,6880483,1445.0


#### merge all the data to get full time series dataframe


In [16]:
timeseries_final = pd.merge(
    global_confirmed_timeseries,
    global_recovered_timeseries,
    how='inner',
    on='date'
)

timeseries_final = pd.merge(
    timeseries_final,
    global_death_timeseries,
    how='inner',
    on='date'
)

timeseries_final

,date,total confirmed,daily new cases,total recovered,daily new recovered,total deaths,daily new deaths
0,1/22/20,557,0.0,30,0.0,17,0.0
1,1/23/20,657,100.0,32,2.0,18,1.0
2,1/24/20,944,287.0,39,7.0,26,8.0
3,1/25/20,1437,493.0,42,3.0,42,16.0
4,1/26/20,2120,683.0,56,14.0,56,14.0
...,...,...,...,...,...,...,...
1138,3/5/23,676024901,56126.0,0,0.0,6877749,148.0
1139,3/6/23,676082941,58040.0,0,0.0,6878115,366.0
1140,3/7/23,676213378,130437.0,0,0.0,6879038,923.0
1141,3/8/23,676392824,179446.0,0,0.0,6880483,1445.0


In [17]:
# total confiremed cases globally
total_confirmed[total_confirmed.columns[-1]].sum()

676570149

In [18]:
# total recovered cases globally
total_recovered[total_recovered.columns[-1]].sum()

0

In [19]:
total_death[total_death.columns[-1]].sum()

6881802

In [20]:
# Data that contain population for each country to calculate confirmed cases/population
df_pop = pd.read_json(
    'https://raw.githubusercontent.com/samayo/country-json/master/src/country-by-population.json')
df_pop

,country,population
0,Afghanistan,37172386
1,Albania,2866376
2,Algeria,42228429
3,American Samoa,55465
4,Andorra,77006
...,...,...
240,Wallis and Futuna,15289
241,Western Sahara,652271
242,Yemen,28498687
243,Zambia,17351822


In [21]:
# some country name has differnt format, so I need to change it to match my first dataset
df_pop.columns = ["Country/Region", "population"]
df_pop = df_pop.replace(to_replace="Russian Federation", value="Russia")
df_pop

,Country/Region,population
0,Afghanistan,37172386
1,Albania,2866376
2,Algeria,42228429
3,American Samoa,55465
4,Andorra,77006
...,...,...
240,Wallis and Futuna,15289
241,Western Sahara,652271
242,Yemen,28498687
243,Zambia,17351822


### country code for geographical analysis


In [22]:
# country code for geographical analysis, Altair need numberical code and Plotly need alfabetical code
# country code and id for later geographical analysis
url = "https://gist.githubusercontent.com/komasaru/9303029/raw/9ea6e5900715afec6ce4ff79a0c4102b09180ddd/iso_3166_1.csv"

country_code = pd.read_csv(url)
country_code = country_code[["English short name", "Alpha-3 code", "Numeric"]]
country_code.columns = ["Country/Region", "code3", "id"]

# change the data for later merging
# if not match the value will be deleted, so we need to make sure each country name from each table has same value
country_code = country_code.replace(
    to_replace="Russian Federation (the)",
    value="United Kingdom"
)

country_code = country_code.replace(
    to_replace="United States (the)",
    value="United States"
)

country_code = country_code.replace(
    to_replace="Viet Nam",
    value="Vietnam"
)

country_code

,Country/Region,code3,id
0,Afghanistan,AFG,4
1,Åland Islands,ALA,248
2,Albania,ALB,8
3,Algeria,DZA,12
4,American Samoa,ASM,16
...,...,...,...
244,Wallis and Futuna,WLF,876
245,Western Sahara*,ESH,732
246,Yemen,YEM,887
247,Zambia,ZMB,894


#### Merge them all


In [23]:
final_df = pd.merge(
    grouped_total_confirmed,
    grouped_total_recovered,
    how="inner",
    on="Country/Region"
)

final_df = pd.merge(
    final_df,
    grouped_total_death,
    how="inner",
    on="Country/Region"
)

final_df = pd.merge(
    final_df,
    df_pop,
    how="inner",
    on="Country/Region"
)

final_df = pd.merge(
    country_code,
    final_df,
    how="inner",
    on="Country/Region"
)

final_df = final_df.sort_values(by="confirmed", ascending=True)
final_df.reset_index(inplace=True, drop=True)
final_df.to_json("new_map.json")

final_df

,Country/Region,code3,id,confirmed,recovered,deaths,population


calculate cases/million and total death rate


In [24]:
final_df["cases/million"] = ((final_df["confirmed"] /
                             final_df['population']) * 1000000).round(2)

final_df["death rate(%)"] = (
    (final_df['deaths'] / final_df["confirmed"]) * 100).round(2)
final_df

,Country/Region,code3,id,confirmed,recovered,deaths,population,cases/million,death rate(%)


In [25]:
country = "Indonesia"
total_all_confirmed = total_confirmed.groupby("Country/Region").sum()
total_all_confirmed.reset_index(inplace=True)

mask = (total_all_confirmed['Country/Region'] == country)

total_all_confirmed = total_all_confirmed.loc[mask]
total_all_confirmed = total_all_confirmed[total_all_confirmed.columns[-1]].sum()

# mask
total_all_confirmed

6738225

function to filter timeseries analysis by country


In [26]:
# use "case" variable just for column name: e.g, case = confirmed, case = deaths
def get_by_country(df: pd.DataFrame, country: str, case: str) -> pd.DataFrame:
    mask = (df["Country/Region"] == country)
    df = df.loc[mask]
    df_country = df.groupby("Country/Region").sum()
    df_country = pd.DataFrame(df[df.columns[4:]].sum())
    df_country.reset_index(inplace=True)
    df_country.columns = ["date", f"value_{case}"]
    df_country[f"daily_new_{case}"] = df_country[f"value_{case}"] - \
        df_country[f"value_{case}"].shift()
    df_country = df_country.fillna(0)
    return df_country

In [28]:
# use function above to get merged dataframe
def get_by_country_merged(total_confirmed, total_death, total_recovered, country):
    # apply to each timeseries
    country_confirmed_tseries = get_by_country(
        total_confirmed, country, "confirmed")
    country_death_tseries = get_by_country(total_death, country, "death")
    country_recovered_tseries = get_by_country(
        total_recovered, country, "recovered")

    # merge them all
    country_timeseries_final = pd.merge(
        country_confirmed_tseries, country_death_tseries, how="inner", on="date")
    country_timeseries_final = pd.merge(
        country_timeseries_final, country_recovered_tseries, how='inner', on="date")
    country_timeseries_final.reset_index(inplace=True)

    return country_timeseries_final

In [29]:
# example for China
country_tseries = get_by_country(total_confirmed, "China", "confirmed")

# it will give me total confirmed cases per day in US, also I can use this for total_deaths and total_recovered too
country_tseries

,date,value_confirmed,daily_new_confirmed
0,1/22/20,548,0.0
1,1/23/20,643,95.0
2,1/24/20,920,277.0
3,1/25/20,1406,486.0
4,1/26/20,2075,669.0
...,...,...,...
1138,3/5/23,4903524,0.0
1139,3/6/23,4903524,0.0
1140,3/7/23,4903524,0.0
1141,3/8/23,4903524,0.0


In [30]:
# example for merged for US
US_ts = get_by_country_merged(
    total_confirmed, total_death, total_recovered, "China")
US_ts

,index,date,value_confirmed,daily_new_confirmed,value_death,daily_new_death,value_recovered,daily_new_recovered
0,0,1/22/20,548,0.0,17,0.0,28,0.0
1,1,1/23/20,643,95.0,18,1.0,30,2.0
2,2,1/24/20,920,277.0,26,8.0,36,6.0
3,3,1/25/20,1406,486.0,42,16.0,39,3.0
4,4,1/26/20,2075,669.0,56,14.0,49,10.0
...,...,...,...,...,...,...,...,...
1138,1138,3/5/23,4903524,0.0,101054,1.0,0,0.0
1139,1139,3/6/23,4903524,0.0,101055,1.0,0,0.0
1140,1140,3/7/23,4903524,0.0,101055,0.0,0,0.0
1141,1141,3/8/23,4903524,0.0,101055,0.0,0,0.0


#### Optional for chart.js geographical analysis

- chart.js doesnt have any map/world map chart, so I need to use 3rd party(highcharts), you can skip this part (2 cells below) if you find it confusing

- that 3rd party use this json file as its data, so I need to mimick its structure


In [31]:
# ref : https://www.highcharts.com/demo/maps/tooltip
chartjs_ccode = pd.read_json(
    "https://cdn.jsdelivr.net/gh/highcharts/highcharts@v7.0.0/samples/data/world-population-density.json")
chartjs_ccode

,code3,name,value,code
0,ABW,Aruba,582.34,AW
1,AFG,Afghanistan,53.08,AF
2,AGO,Angola,23.11,AO
3,ALB,Albania,104.97,AL
4,AND,Andorra,164.43,AD
...,...,...,...,...
208,WSM,Samoa,68.95,WS
209,YEM,"Yemen, Rep.",52.25,YE
210,ZAF,South Africa,46.18,ZA
211,ZMB,Zambia,22.32,ZM


In [36]:
# dump it to external json_file
import json
chartjs_map = final_df[["code3", "Country/Region", "confirmed"]]
chartjs_map.columns = ["code3", "name", "value"]
chart_json = chartjs_map.to_dict('records')

with open('chart_js.json', 'w') as fout:
    json.dump(chart_json, fout)

chart_json

[]

## 2. Altair Plot


In [38]:
import altair as alt

# Plot Altair 1: Per country total cases and cases/million populations
source = final_df

# base configuration
base = alt.Chart(source).encode(
    alt.X('Country/Region:N', sort=None),
    tooltip=["Country/Region", "confirmed", "cases/million"]
).properties(
    height=500,
    title="Total Confirmed Cases/Country"
)

# bar title configuration

# bar chart
bar = base.mark_bar(color="#5276A7").encode(
    alt.Y('confirmed:Q', axis=alt.Axis(titleColor='#5276A7'))
)

# point for cases/million and its axis
point = base.mark_circle(size=60, color='red').encode(
    alt.Y('cases/million:Q', axis=alt.Axis(titleColor="red"))
)

# merge the plot
alt.layer(bar, point).resolve_scale(y="independent")

c:\Users\Makara PC\.conda\envs\ml-practice\Lib\site-packages\altair\utils\core.py:230: UserWarning:

I don't know how to infer vegalite type from 'empty'.  Defaulting to nominal.



alt.LayerChart(...)